In [147]:
import random

In [190]:
class value:
    def __init__(self, data, label = "V", backprop = lambda : None, grad = 0.0):
        self.data, self.label, self.backprop, self.grad = data, label, backprop, grad
    def __add__(self, other):
        out =  value(self.data + other.data)
        def backprop():
            self.grad += out.grad
            other.grad += out.grad
            self.backprop()
            other.backprop()
        out.backprop = backprop
        return out
    def __mul__(self, other):
        out =  value(self.data * other.data)
        def backprop():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
            self.backprop()
            other.backprop()
        out.backprop = backprop
        return out
    def __radd__(self, other):
        return self.__add__(value(other))
    def __repr__(self):
        return f"{self.label} : {self.data}"
    def __pow__(self, power : int):
        out = value(self.data ** power)
        def backprop():
            self.grad+= out.grad * (power * (self.data ** (power - 1)))
            self.backprop()
        out.backprop = backprop
        return out
    def __sub__(self, other):
        return self + (value(-1) * other)
    def __truediv__(self, other):
        return self * (other ** -1)
    def ReLU(self):
        out = value(0 if self.data < 0 else self.data)
        def backprop():
            self.grad += out.grad * (0 if self.data < 0 else 1)
            self.backprop()
        out.backprop = backprop
        return out


def Numerical2Value(numericalArr : list) -> list[value]:
    return [value(numerical) for numerical in numericalArr]

def mse(predictedVec : list[value], actualVec : list[value]) -> value:
    simpleError = sum([(prediction - actual) ** 2 for prediction , actual in zip(predictedVec, actualVec)]) / value(len(actualVec))
    return simpleError

In [201]:
class neuron:
    def __init__(self, inputSize : int):
        self.b = value(random.uniform(-1, 1))
        self.w = [value(random.uniform(-1, 1)) for _ in range(inputSize)]
    def __call__(self, inputs):
        weightedSum = sum([xi * wi for xi, wi in zip(inputs, self.w)])
        addedBias = self.b + weightedSum
        activated = addedBias.ReLU()
        return activated
    def getParams(self):
        return [self.b] + self.w

class layer:
    def __init__(self, inputSize, outputSize):
        self.neurons = [neuron(inputSize) for _ in range(outputSize)]
    def __call__(self, inputs):
        return [n(inputs) for n in self.neurons]
    def getParams(self):
        return [params for neuron in self.neurons for params in neuron.getParams()]

class network:
    def __init__(self, layerDims : list):
        self.layers = []
        for i in range(len(layerDims) - 1):
            self.layers.append(layer(layerDims[i], layerDims[i + 1]))
    def __call__(self, inputVec : value):
        outputVec = inputVec
        for layer in self.layers:
            outputVec = layer(outputVec)
        return outputVec
    def getParams(self) -> list[value]:
        return [params for layer in self.layers for params in layer.getParams()]


In [246]:
def trainer(network, loss):
    loss.grad = 1.0
    def backpropLoss():
        loss.backprop()

    def adjustParams(lr = 0.001):
        for p in network.getParams():
            p.data -= p.grad * lr

    def flushGradients():
        for p in network.getParams():
            p.grad = 0.0
    
    flushGradients()
    backpropLoss()
    adjustParams()
    

n = network([2, 2, 2, 1])
